In [8]:
import pandas as pd
import json
import ast
import hashlib
import uuid

def parse_json_like_string(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return value

def generate_quoc_gia_id(name_dict, namespace_uuid):
    if isinstance(name_dict, dict):
        common_name = name_dict.get('common', '')
        official_name = name_dict.get('official', '')
        combined_string = f"{common_name}{official_name}"
        return uuid.uuid5(namespace_uuid, combined_string)
    return None 
    
# with open('raw/countries.json', 'r') as f:
#         data = json.load(f)

# df = pd.DataFrame(data)
# df.fillna('', inplace=True)
# df

df = pd.read_csv('foundation/countries.csv', sep=',')
df = df.map(parse_json_like_string)
namespace_uuid = uuid.UUID("9a5963f8-5a5c-4b8c-aa46-1068af074546")

# Transformations
df['capital'] = df['capital'].apply(lambda x: "|".join(x) if isinstance(x, list) else x)
df['altSpellings'] = df['altSpellings'].apply(lambda x: "|".join(x) if isinstance(x, list) else x)
df['kinh_do'] = df['latlng'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['vi_do'] = df['latlng'].apply(lambda x: x[1] if isinstance(x, list) else x)
df['su_dung_tieng_anh'] = df['languages'].notna()
df['maps'] = df['maps'].apply(lambda x: x.get('googleMaps', '') if isinstance(x, dict) else '')
df['timezones'] = df['timezones'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['flags'] = df['flags'].apply(lambda x: x.get('svg', '') if isinstance(x, dict) else '')

df['quoc_gia_id'] = df['name'].apply(lambda x: generate_quoc_gia_id(x, namespace_uuid))
currency_df = df[['quoc_gia_id', 'currencies']].copy()
currency_df = currency_df.explode('currencies')
currency_df

# df = df.map(parse_json_like_string)

# # Transformations
# df['capital'] = df['capital'].apply(lambda x: "|".join(x) if isinstance(x, list) else x)
# df['altSpellings'] = df['altSpellings'].apply(lambda x: "|".join(x) if isinstance(x, list) else x)
# df['latlng'] = df['latlng'].apply(lambda x: f"kinh_do={x[0]}|vi_do={x[1]}" if isinstance(x, list) else x)
# df['su_dung_tieng_anh'] = df['languages'].notna()
# df['maps'] = df['maps'].apply(lambda x: x.get('googleMaps', '') if isinstance(x, dict) else '')
# df['timezones'] = df['timezones'].apply(lambda x: x[0] if isinstance(x, list) else x)
# df['flags'] = df['flags'].apply(lambda x: x.get('svg', '') if isinstance(x, dict) else '')

# df.rename(columns={
# 'name.common': 'ten',
# 'name.official': 'ten_chinh_thuc',
# 'ccn3': 'ma_quoc_gia',
# 'cca3': 'ma_quoc_gia_3',
# 'independent': 'doc_lap',
# 'status': 'tinh_trang',
# 'capital': 'thu_do',
# 'altSpellings': 'ten_khac',
# 'region': 'vung',
# 'area': 'dien_tich',
# 'maps.googleMaps': 'ban_do',
# 'timezones': 'mui_gio',
# 'continents': 'luc_dia',
# 'flags.svg': 'co',
# 'startOfWeek': 'bat_dau_tuan'
# }, inplace=True)

# df

# Create new columns
# df['quoc_gia_id'] = df.apply(lambda row: hashlib.md5(f"{row['ten']} {row['ten_chinh_thuc']}".encode('utf-8')).hexdigest(), axis=1)


,quoc_gia_id,currencies
0,47f382ee-1527-5e24-b312-0f25515c9cd8,SHP
1,48645d4d-30c7-5a93-a90e-4694df666fa3,XCD
2,f286f023-a63a-5d0b-b7b1-ab7132a95bbf,CHF
3,f6de9b40-f367-55a3-a8a1-f5bb10c32d2f,SLL
4,2052c9b8-d256-525e-a50c-f48134767822,HUF
...,...,...
245,11f28438-b0c4-56d6-9383-47529a23e2f0,EUR
246,6882a667-b8d2-5e31-8c46-2d33614953e5,ILS
247,452b3870-8181-5cf1-bbcd-f7a3fbb6f64b,NZD
248,5f080ddf-58d1-5c58-8f71-ddfe87c56b48,NIO
